# Check GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# List all NVIDIA GPUs as avaialble in this computer (or Colab's session)
!nvidia-smi -L

In [ ]:
import sys
print( f"Python {sys.version}\n" )

import numpy as np
print( f"NumPy {np.__version__}\n" )

import matplotlib.pyplot as plt
%matplotlib inline

# For image resizing
import cv2 as cv
print( f"OpenCV {cv.__version__}\n" )

import tensorflow as tf
print( f"TensorFlow {tf.__version__}" )
print( f"tf.keras.backend.image_data_format() = {tf.keras.backend.image_data_format()}" )

# Count the number of GPUs as detected by tensorflow
gpus = tf.config.list_physical_devices('GPU')
print( f"TensorFlow detected { len(gpus) } GPU(s):" )
for i, gpu in enumerate(gpus):
  print( f".... GPU No. {i}: Name = {gpu.name} , Type = {gpu.device_type}" )

## Prepare Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
import os
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, roc_curve
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
data_path = pathlib.Path(r"/content/drive/MyDrive/Colab Notebooks/CNN (1)/data")

all_images = list(data_path.glob(r'*/*.jpg')) + list(data_path.glob(r'*/*.jpeg')) + list(data_path.glob(r'*/*.png'))

images = []
labels = []

for item in all_images:
    path = os.path.normpath(item)
    splits = path.split(os.sep)
    if 'GT' not in splits[-2]:
        images.append(item)
        label = splits[-2]
        labels.append(label)

In [ ]:
image_pathes = pd.Series(images).astype(str)
labels = pd.Series(labels)

dataframe =pd.concat([image_pathes, labels], axis=1)

dataframe.columns = ['images', 'labels']

dataframe.head()

In [ ]:
dataframe.groupby('labels').count()

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(15,10), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(dataframe.images[i]))
    ax.set_title(dataframe.labels[i])
plt.show()

In [ ]:
shuffled_dataframe = dataframe.sample(frac = 1)

In [ ]:
all_train, test = train_test_split(shuffled_dataframe, test_size=0.2, random_state=42)
train, val = train_test_split(all_train, test_size=0.3, random_state=42)

In [ ]:
test.head()

In [ ]:
training_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
training_generator = training_data_gen.flow_from_dataframe(dataframe=train,
                                                          x_col='images', y_col='labels',
                                                          target_size=(224, 224),
                                                          color_mode='rgb',
                                                          class_mode='categorical',
                                                          batch_size=64)

val_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)
validation_generator = val_data_gen.flow_from_dataframe(dataframe=val,
                                                       x_col='images', y_col='labels',
                                                       target_size=(224, 224),
                                                       color_mode='rgb',
                                                       class_mode='categorical',
                                                       batch_size=64)

test_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)
test_generator = test_data_gen.flow_from_dataframe(dataframe=test,
                                                  x_col='images', y_col='labels',
                                                  target_size=(224, 224),
                                                  color_mode='rgb',
                                                  class_mode='categorical',
                                                  batch_size=64,
                                                  shuffle=False)

In [ ]:
label_map = test_generator.class_indices
label_map

In [ ]:
test_generator.class_indices.keys()

# Import VGG16 Model

 VGG16 Base Model 

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = True
    
from tensorflow.keras import layers 
from tensorflow.keras import Model 

In [ ]:
y = layers.Flatten()(base_model.output)
y = layers.Dense(77, activation='softmax')(y)
base_model = tf.keras.models.Model(base_model.input, y)
base_model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

In [ ]:
vgg_basehist = base_model.fit(training_generator, validation_data = validation_generator, epochs = 50)

In [ ]:
base_model_train_loss = vgg_basehist.history['loss']
base_model_val_loss =vgg_basehist.history['val_loss']

plt.plot(vgg_basehist.epoch, base_model_train_loss, label='Trainnig Loss')
plt.plot(vgg_basehist.epoch, base_model_val_loss, label='Validation Loss')
plt.grid(True)
plt.legend()

In [ ]:
base_model_train_acc =vgg_basehist.history['acc']
base_model_val_acc = vgg_basehist.history['val_acc']

plt.plot(vgg_basehist.epoch, base_model_train_acc, label='Trainnig Accuracy')
plt.plot(vgg_basehist.epoch, base_model_val_acc, label='Validation Accuracy')
plt.grid(True)
plt.legend()

In [ ]:
base_model.evaluate(test_generator)

In [ ]:
# Switch Dict and Value
label_map_switch = {}
for k, v in label_map.items():
    label_map_switch[v] = k
label_map_switch

In [ ]:
predictions = base_model.predict(test_generator)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability
predicted_label_batch =list(map(label_map_switch.get, pred_labels.tolist()))

In [ ]:
test_dup = test.copy()
test_dup = test_dup.reset_index(drop=True)
test_dup["predict"] = pd.DataFrame(predicted_label_batch)
test_dup.head()

In [ ]:
y_pred = [label_map_switch[k] for k in pred_labels]
print(classification_report(test.labels, y_pred))

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=5, figsize=(20,20), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_dup.images[i]))
    color = "green" if test_dup.predict[i] == test_dup.labels[i] else "red"
    ax.set_title(test_dup.labels[i],color=color)
plt.show()

VGG16 Finetuning

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

vgg_base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [ ]:
for layer in vgg_base_model.layers:
    layer.trainable = False
    
from tensorflow.keras import layers 
from tensorflow.keras import Model 

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(vgg_base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final softmax layer with 77 nodes for classification output
x = layers.Dense(77, activation='softmax')(x)

vgg_newmodel = tf.keras.models.Model(vgg_base_model.input, x)

vgg_newmodel.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

In [ ]:
vgghist = vgg_newmodel.fit(training_generator, validation_data = validation_generator, epochs = 50)

In [ ]:
model_train_loss = vgghist.history['loss']
model_val_loss = vgghist.history['val_loss']

plt.plot(vgghist.epoch, model_train_loss, label='Trainnig Loss')
plt.plot(vgghist.epoch, model_val_loss, label='Validation Loss')
plt.grid(True)
plt.legend()

In [ ]:
train_acc = vgghist.history['acc']
val_acc = vgghist.history['val_acc']

plt.plot(vgghist.epoch, train_acc, label='Trainnig Accuracy')
plt.plot(vgghist.epoch, val_acc, label='Validation Accuracy')
plt.grid(True)
plt.legend()

In [ ]:
vgg_newmodel.evaluate(test_generator)

In [ ]:
# Switch Dict and Value
label_map_switch = {}
for k, v in label_map.items():
    label_map_switch[v] = k
label_map_switch

In [ ]:
predictions = vgg_newmodel.predict(test_generator)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability
predicted_label_batch =list(map(label_map_switch.get, pred_labels.tolist()))

In [ ]:
test_dup = test.copy()
test_dup = test_dup.reset_index(drop=True)
test_dup["predict"] = pd.DataFrame(predicted_label_batch)
test_dup.head()

In [ ]:
y_pred = [label_map_switch[k] for k in pred_labels]
print(classification_report(test.labels, y_pred))

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=5, figsize=(20,20), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_dup.images[i]))
    color = "green" if test_dup.predict[i] == test_dup.labels[i] else "red"
    ax.set_title(test_dup.labels[i],color=color)
plt.show()

# Import EfficientNet

EfficientNet Base Model

In [ ]:
pip install efficientnet

In [ ]:
import efficientnet.keras as efn
base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = True
    
from tensorflow.keras import layers 
from tensorflow.keras import Model 

In [ ]:
y = layers.Flatten()(base_model.output)
y = layers.Dense(77, activation='softmax')(y)
base_model = tf.keras.models.Model(base_model.input, y)
base_model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

In [ ]:
efn_basehist = base_model.fit(training_generator, validation_data = validation_generator, epochs = 50)

In [ ]:
base_model_train_loss = efn_basehist.history['loss']
base_model_val_loss = efn_basehist.history['val_loss']

plt.plot(efn_basehist.epoch, base_model_train_loss, label='Trainnig Loss')
plt.plot(efn_basehist.epoch, base_model_val_loss, label='Validation Loss')
plt.grid(True)
plt.legend()

In [ ]:
base_model_train_acc =efn_basehist.history['acc']
base_model_val_acc = efn_basehist.history['val_acc']

plt.plot(efn_basehist.epoch, base_model_train_acc, label='Trainnig Accuracy')
plt.plot(efn_basehist.epoch, base_model_val_acc, label='Validation Accuracy')
plt.grid(True)
plt.legend()

In [ ]:
base_model.evaluate(test_generator)

In [ ]:
predictions = base_model.predict(test_generator)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability
predicted_label_batch =list(map(label_map_switch.get, pred_labels.tolist()))

In [ ]:
test_dup = test.copy()
test_dup = test_dup.reset_index(drop=True)
test_dup["predict"] = pd.DataFrame(predicted_label_batch)
test_dup.head()

In [ ]:
y_pred = [label_map_switch[k] for k in pred_labels]
print(classification_report(test.labels, y_pred))

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=5, figsize=(20,20), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_dup.images[i]))
    color = "green" if test_dup.predict[i] == test_dup.labels[i] else "red"
    ax.set_title(test_dup.labels[i],color=color)
plt.show()

EfficientNet Finetunig

In [ ]:
import efficientnet.keras as efn
base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    
from tensorflow.keras import layers 
from tensorflow.keras import Model 

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final softmax layer with 77 nodes for classification output
x = layers.Dense(77, activation='softmax')(x)

efn_newmodel = tf.keras.models.Model(base_model.input, x)

efn_newmodel.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

In [ ]:
efnhist = efn_newmodel.fit(training_generator, validation_data = validation_generator, epochs = 50)

In [ ]:
model_train_loss = efnhist.history['loss']
model_val_loss = efnhist.history['val_loss']

plt.plot(efnhist.epoch, model_train_loss, label='Trainnig Loss')
plt.plot(efnhist.epoch, model_val_loss, label='Validation Loss')
plt.grid(True)
plt.legend()

In [ ]:
train_acc = efnhist.history['acc']
val_acc = efnhist.history['val_acc']

plt.plot(efnhist.epoch, train_acc, label='Trainnig Accuracy')
plt.plot(efnhist.epoch, val_acc, label='Validation Accuracy')
plt.grid(True)
plt.legend()

In [ ]:
efn_newmodel.evaluate(test_generator)

In [ ]:
# Switch Dict and Value
label_map_switch = {}
for k, v in label_map.items():
    label_map_switch[v] = k
label_map_switch

In [ ]:
predictions = efn_newmodel.predict(test_generator)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability
predicted_label_batch =list(map(label_map_switch.get, pred_labels.tolist()))

In [ ]:
test_dup = test.copy()
test_dup = test_dup.reset_index(drop=True)
test_dup["predict"] = pd.DataFrame(predicted_label_batch)
test_dup.head()

In [ ]:
y_pred = [label_map_switch[k] for k in pred_labels]
print(classification_report(test.labels, y_pred))

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=5, figsize=(20,20), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_dup.images[i]))
    color = "green" if test_dup.predict[i] == test_dup.labels[i] else "red"
    ax.set_title(test_dup.labels[i],color=color)
plt.show()

# Import ResNet50 Model

ResNet50 Base Model

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [ ]:
for layer in base_model.layers:
    layer.trainable = True
    
from tensorflow.keras import layers 
from tensorflow.keras import Model 

In [ ]:
y = layers.Flatten()(base_model.output)
y = layers.Dense(77, activation='softmax')(y)
base_model = tf.keras.models.Model(base_model.input, y)
base_model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

In [ ]:
resnet_basehist = base_model.fit(training_generator, validation_data = validation_generator, epochs = 50)

In [ ]:
base_model_train_loss = resnet_basehist.history['loss']
base_model_val_loss = resnet_basehist.history['val_loss']

plt.plot(resnet_basehist.epoch, base_model_train_loss, label='Trainnig Loss')
plt.plot(res_netbasehist.epoch, base_model_val_loss, label='Validation Loss')
plt.grid(True)
plt.legend()

In [ ]:
base_model_train_acc = resnet_basehist.history['acc']
base_model_val_acc = resnet_basehist.history['val_acc']

plt.plot(resnet_basehist.epoch, base_model_train_acc, label='Trainnig Accuracy')
plt.plot(resnet_basehist.epoch, base_model_val_acc, label='Validation Accuracy')
plt.grid(True)
plt.legend()

In [ ]:
base_model.evaluate(test_generator)

In [ ]:
predictions = base_model.predict(test_generator)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability
predicted_label_batch =list(map(label_map_switch.get, pred_labels.tolist()))

In [ ]:
test_dup = test.copy()
test_dup = test_dup.reset_index(drop=True)
test_dup["predict"] = pd.DataFrame(predicted_label_batch)
test_dup.head()

In [ ]:
y_pred = [label_map_switch[k] for k in pred_labels]
print(classification_report(test.labels, y_pred))

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=5, figsize=(20,20), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_dup.images[i]))
    color = "green" if test_dup.predict[i] == test_dup.labels[i] else "red"
    ax.set_title(test_dup.labels[i],color=color)
plt.show()

ResNet50 Finetuning

In [ ]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    
from tensorflow.keras import layers 
from tensorflow.keras import Model 

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final softmax layer with 77 node for classification output
x = layers.Dense(77, activation='softmax')(x)

resnet_newmodel = tf.keras.models.Model(base_model.input, x)

resnet_newmodel.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

In [ ]:
resnethist = resnet_newmodel.fit(training_generator, validation_data = validation_generator, epochs = 100)

In [ ]:
model_train_loss = resnethist.history['loss']
model_val_loss = resnethist.history['val_loss']

plt.plot(resnethist.epoch, model_train_loss, label='Trainnig Loss')
plt.plot(resnethist.epoch, model_val_loss, label='Validation Loss')
plt.grid(True)
plt.legend()

In [ ]:
train_acc = resnethist.history['acc']
val_acc = resnethist.history['val_acc']

plt.plot(resnethist.epoch, train_acc, label='Trainnig Accuracy')
plt.plot(resnethist.epoch, val_acc, label='Validation Accuracy')
plt.grid(True)
plt.legend()

In [ ]:
resnet_newmodel.evaluate(test_generator)

In [ ]:
# Switch Dict and Value
label_map_switch = {}
for k, v in label_map.items():
    label_map_switch[v] = k
label_map_switch

In [ ]:
predictions = resnet_newmodel.predict(test_generator)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability
predicted_label_batch =list(map(label_map_switch.get, pred_labels.tolist()))

In [ ]:
test_dup = test.copy()
test_dup = test_dup.reset_index(drop=True)
test_dup["predict"] = pd.DataFrame(predicted_label_batch)
test_dup.head()

In [ ]:
y_pred = [label_map_switch[k] for k in pred_labels]
print(classification_report(test.labels, y_pred))

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=5, figsize=(20,20), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_dup.images[i]))
    color = "green" if test_dup.predict[i] == test_dup.labels[i] else "red"
    ax.set_title(test_dup.labels[i],color=color)
plt.show()

# Self-Build Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (224, 224, 4)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(77, activation=tf.nn.softmax)
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(training_generator, epochs = 50, validation_data=validation_generator)

In [ ]:
model_train_loss = history.history['loss']
model_val_loss = history.history['val_loss']

plt.plot(history.epoch, model_train_loss, label='Trainnig Loss')
plt.plot(history.epoch, model_val_loss, label='Validation Loss')
plt.grid(True)
plt.legend()

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(history.epoch, train_acc, label='Trainnig Accuracy')
plt.plot(history.epoch, val_acc, label='Validation Accuracy')
plt.grid(True)
plt.legend()

In [ ]:
model.evaluate(test_generator)

In [ ]:
# Switch Dict and Value
label_map_switch = {}
for k, v in label_map.items():
    label_map_switch[v] = k
label_map_switch

In [ ]:
predictions = model.predict(test_generator)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability
predicted_label_batch =list(map(label_map_switch.get, pred_labels.tolist()))

In [ ]:
test_dup = test.copy()
test_dup = test_dup.reset_index(drop=True)
test_dup["predict"] = pd.DataFrame(predicted_label_batch)
test_dup.head()

In [ ]:
y_pred = [label_map_switch[k] for k in pred_labels]
print(classification_report(test.labels, y_pred))

In [ ]:
fig, axes = plt.subplots(nrows=10, ncols=5, figsize=(20,20), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_dup.images[i]))
    color = "green" if test_dup.predict[i] == test_dup.labels[i] else "red"
    ax.set_title(test_dup.labels[i],color=color)
plt.show()